In [2]:
import numpy as np
import pyfits
import sys
import scipy as sp
import scipy.spatial as ss
import math
import astropy as ap
from astropy.cosmology import WMAP7
from random import sample
import math
import pickle
import matplotlib.pyplot as plt
import os

os.chdir('/Users/nitya/Work/Data/Project1/measurements')
print(os.getcwd())


/Users/nitya/Work/Data/Project1/measurements


In [3]:
# d = np.loadtxt('edp')
# np.shape(d)


file = open(r'ETP.pkl','rb')
d = pickle.load(file, encoding ='latin1')
file.close()

print(np.shape(d))

(78174,)


In [3]:
nsaid = np.asarray(d['NSAID'])
z = np.asarray(d['Z'])
ra = np.asarray(d['RA'])
dec = np.asarray(d['DEC'])

In [4]:
# x = np.asarray([comdis[i]*(math.cos(math.radians(dec[i])))*(math.cos(math.radians(ra[i])))for i in range(len(z))])
# y = np.asarray([(comdis[i])*(math.cos(math.radians(dec[i])))*(math.sin(math.radians(ra[i]))) for i in range(len(z))])
# zed = np.asarray([(comdis[i])*(math.sin(math.radians(dec[i]))) for i in range(len(z))])

In [5]:
def coords(ra,dec,z):
        dist = np.array([WMAP7.comoving_distance(z[i]).value for i in range(len(z))])
        
        x = dist*np.cos(dec*np.pi/180.0)*np.cos(ra*np.pi/180.0)
        y = dist*np.cos(dec*np.pi/180.0)*np.sin(ra*np.pi/180.0)
        zed = dist*np.sin(dec*np.pi/180.0)
        return x,y,zed
b = coords(ra,dec,z)
x, y, zed = coords(ra,dec,z)

In [6]:
points = np.column_stack((x,y,zed))
tree = ss.KDTree(points)


In [7]:
#window aperture: ap: 0.5,1.0 Mpc; window size: w: +/- 500 km/s, 1000 km/s
ap = 0.5
w = 1000

#But first we construct
s = ((ap)**2.0 + (w/70.0)**2.0)**0.5
ns = tree.query_ball_point(points,s)


In [8]:

#Now, to get the number of neighbors within my projected cylinder:
nc = []
#neighbors that survive...
nei = []
for i in range(len(points)):
    count = 0
    surv = []
    for j in range(len(ns[i])):
        g =(np.sin(dec[i]*np.pi/180.0))*(np.sin(dec[ns[i][j]]*np.pi/180.0))
        p =(np.cos(dec[i]*np.pi/180.0))*(np.cos(dec[ns[i][j]]*np.pi/180.0))*(np.cos((ra[i] - ra[ns[i][j]])*np.pi/180.0))
        ang_sep = g+p
        d_transverse = (300000.0/70.0)*(z[i]+z[ns[i][j]])*((abs((1 -ang_sep)/2.0))**0.5)
        d_los =  (300000.0)* abs(z[i] - z[ns[i][j]])
        if d_transverse <= ap and d_los <= w:
           count = count+1
           surv.append(nsaid[ns[i][j]])
    nc.append(count)
    nei.append(surv)

print(len(nc), len(ns), s)

78174 78174 14.29446160766684


In [9]:

stuff = np.column_stack((nsaid, nc))
afile = open(r'nsaid_env.pkl', 'wb')
pickle.dump(stuff, afile)
afile.close()

stuff1 = np.column_stack((nsaid,nei))
afile = open(r'nsaid_neighbors.pkl', 'wb')
pickle.dump(stuff1, afile)
afile.close()

In [17]:

# n = 3
# neighbours = tree.query(points,k = n+1)
# #The distance to the nth neighbour
# dist = neighbours[0][:,n]
# #The projected distances: Let's first get the index of the n'th neighbour
# n_index = neighbours[1][:,n]
# print(len(dist), dist[0:10])
# stuff2 = np.column_stack((nsaid,dist))
# afile = open(r'd3_neighbor.pkl', 'wb')
# pickle.dump(stuff2, afile)
# afile.close()

In [18]:

# dist_proj = []

# for i in range(len(z)):
#     g =(np.sin(dec[i]*np.pi/180.0))*(np.sin(dec[n_index[i]]*np.pi/180.0))
#     p =(np.cos(dec[i]*np.pi/180.0))*(np.cos(dec[n_index[i]]*np.pi/180.0))*(np.cos((ra[i] - ra[n_index[i]])*np.pi/180.0))
#     ang_sep = g+p
#     d_transverse = (300000.0/70.0)*(z[i]+z[n_index[i]])*((abs((1 -ang_sep)/2.0))**0.5)
#     d_los =  (300000.0)* abs(z[i] - z[n_index[i]])
#     dist_proj.append(d_transverse)
    
# stuff3 = np.column_stack((nsaid,dist_proj))
# afile = open(r'dp3_neighbor.pkl', 'wb')
# pickle.dump(stuff3, afile)
# afile.close()

